In [32]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from scipy import stats
import numpy as np
import warnings
warnings.simplefilter('ignore')
%matplotlib inline

In [33]:
df = pd.read_excel('df_cinema_without.xlsx')

In [34]:
films = pd.read_excel('prem_year.xlsx')

In [ ]:
#Оставим в датасете только фильмы
df = df[df['movie_name'].isin(films['movie_name'])]

In [ ]:
df.info()

## Анализ по теме исследования

In [ ]:
#Присоединение колонки дня недели премьеры, дня премьеры, количества билетов на фильм
unique = df.sort_values(by=['session_date','movie_name']).drop_duplicates(subset='movie_name')
unique = unique.assign(prem_day = lambda x: x['session_date'].dt.day_name())
unique = unique.assign(prem_day_of_month = lambda x: x['session_date'].dt.day)
unique = unique[['movie_name','prem_day','prem_day_of_month','session_date']]
unique = unique.rename(columns={'session_date':'premier_date'})

unique  = unique.merge(df.groupby('movie_name').count()['order_id'].reset_index().rename(columns={'order_id':'ticks_count'}), how='inner', on='movie_name')
unique.sort_values(by='ticks_count')

df = df.merge(unique, on='movie_name',how='inner')

In [ ]:
#распределение фильмов по категориям
d2 = {}  # фильм: кассовый сбор
s = set()
for i, data in df.iterrows():
    film = data['movie_name']
    d2[film] = d2.get(film, 0) + data['sales_in_cu']
a = []
for i in d2:
    a += [[i, d2[i]]]
groups = pd.DataFrame(a, columns =['movie_name', 'sum_profit'])
groups['success'] = np.nan * len(groups)
quantile_25 = groups.sum_profit.quantile(q=0.25)
quantile_75 = groups.sum_profit.quantile(q=0.75)
quantile_99 = groups.sum_profit.quantile(q=0.99)
for i, data in groups.iterrows():
    sum_profit = data['sum_profit']
    if sum_profit < quantile_25:
        groups.success[i] = 1
    if sum_profit >= quantile_25 and sum_profit < quantile_75:
        groups.success[i] = 2
    if sum_profit >= quantile_75 and sum_profit < quantile_99:
        groups.success[i] = 3
    if sum_profit >= quantile_99:
        groups.success[i] = 4
        
df = df.merge(groups.drop('sum_profit',axis=1), on='movie_name',how='inner')

In [ ]:
# Кэффициент корреляции между фильмами, вышедшими в четверг и его долей в общих кассовых сборах(слишком сложная интерпретация)

# coefs = []
# for n in tqdm(range(1170)):
#     ts = df.groupby('movie_name').sum()['sales_in_cu'].sort_values(ascending=False)[:n]
#     data = df[~df['movie_name'].isin(ts.index)].groupby('prem_day').sum()['sales_in_cu'].sort_values(ascending=False) 
#     data /= unique[~unique['movie_name'].isin(ts.index)].groupby('prem_day').count()['movie_name']
    
#     coefs.append(data['Thursday']/(data['Thursday']+data['Wednesday']+data['Monday']+data['Tuesday']+data['Sunday']+data['Friday']+data['Saturday']))
    
# coef = pd.DataFrame([[i for i in range(1170)],coefs], index=['n','coef']).T
# coef = coef.assign(coefdir = lambda x: 1/x['coef'] )
# sns.lmplot(data = coef, x = 'n', y ='coef')
# np.corrcoef(x=coef['n'], y=coef['coef'])[0][1]

In [ ]:
sns.heatmap(df.corr())

In [ ]:
#кол-во купленных билетов до карантина
data = df[df['session_date']<='2020-03-25']
plt.plot(data.assign(dayofweek = lambda x: x['session_date'].dt.day_name()).groupby('dayofweek').count()['order_id'])

In [ ]:
#кол-во купленных билетов после ослабления карантина
data = df[df['session_date']>'2020-07-31']
plt.plot(data.assign(dayofweek = lambda x: x['session_date'].dt.day_name()).groupby('dayofweek').count()['order_id'])

## Способы убрать выбросы

#### Был вариант убрать фильмы с маленьким рейтингом, количеству проданных билетов, но остановились на фильмах, выпущенных до 20-го года из-за наибольшей объективности способа(т.к. считаем по премьерам фильмов, до этого не имевших фанатов, рейтинг и т.п.)

In [ ]:
unique.ticks_count.plot.box(showmeans=True)

In [ ]:
unique.ticks_count.plot.box(showfliers=0)

In [ ]:
outs = df[df['success']<2]
outs

In [ ]:
outs.groupby('movie_name').sum()['sales_in_cu'].sort_values(ascending=0)

In [ ]:
outs = df[df['ticks_count']<100]
outs

### Разделяем на датасет на фильмы премьера которых была в 20м году, и на те, которые вышли ранее 19-го года

In [ ]:
films_20_year = films[films['year']==2020]
films_before_20_year = films[films['year']<2019]

In [ ]:
gip1_df =df[df['movie_name'].isin(films_20_year['movie_name'])]
gip1_df = gip1_df[gip1_df['premier_date']+ pd.Timedelta(14,unit='D') >= gip1_df['session_date']]

In [ ]:
gip2_df =df[df['movie_name'].isin(films_before_20_year['movie_name'])]
gip2_df = gip2_df[gip2_df['premier_date']+ pd.Timedelta(14,unit='D') >= gip2_df['session_date']]

In [ ]:
#graphs for gif
# for n in tqdm(range(0,104,3)):
#     ts = df.groupby('movie_name').sum()['sales_in_cu'].sort_values(ascending=False)[:n]
#     data = df[~df['movie_name'].isin(ts.index)].groupby(['prem_day','success']).sum()['sales_in_cu'].reset_index()
#     fig = sns.barplot(data = data, x='prem_day',y=data['sales_in_cu'],hue='success',palette='viridis_r').get_figure()
#     plt.title(str(n))
#     plt.savefig(f'{n}.png')
#     plt.clf() 
# None

###  Первая гипотеза(Среда и четверг наиболее оптимальные дни для выпуска фильма в прокат)

In [ ]:
plt.figure(figsize=(15,8))
#sns.boxplot(data=df,x='prem_day',y='sales_in_cu',showmeans=1,showfliers=0)
sns.boxplot(data=gip1_df.groupby('movie_name').sum()['sales_in_cu'].reset_index().merge(
    unique[['prem_day','movie_name']],how='inner',on='movie_name'),
            x='prem_day',y='sales_in_cu',showfliers=0,showmeans=1, palette='autumn_r',
            order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])

In [ ]:
sns.countplot(data=gip1_df.drop_duplicates(subset='movie_name'), x='prem_day')

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=gip1_df.drop_duplicates(subset='movie_name'),x='prem_day',hue='success',
              order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
None

In [ ]:
gip1 =gip1_df.groupby('movie_name').sum()['sales_in_cu'].reset_index().merge(unique[['prem_day','movie_name']],how='inner',on='movie_name')
groupa = gip1[gip1['prem_day'].isin(['Thursday','Wednesday'])]['sales_in_cu']
groupb = gip1[~gip1['prem_day'].isin(['Thursday','Wednesday'])]['sales_in_cu']
pvalue_1 = stats.ttest_ind(groupa, groupb,equal_var=0,alternative='greater')[1]
print('p-value для первой гипотезы:',pvalue_1)
print('Не отклоняем нулевую гипотезу' if pvalue_1 > 0.05 else 'Отклоняем нулевую гипотезу')

In [ ]:
#Среда - оптимальный день(хиквадрат тест)
data = gip1_df.drop_duplicates(subset='movie_name')
data = data[data['prem_day']!='Thursday']
data['prem_day'] = data['prem_day'].map(
    {'Friday':0,'Wednesday':1, 'Thursday':0,'Monday':0,'Saturday':0,'Sunday':0,'Tuesday':0})
data[['prem_day','success']].to_csv('gip1_1.csv')

In [ ]:
#Четверг - оптимальный день(хиквадрат тест)
data = gip1_df.drop_duplicates(subset='movie_name')
data['prem_day'] = data['prem_day'].map(
    {'Friday':0,'Wednesday':0, 'Thursday':1,'Monday':0,'Saturday':0,'Sunday':0,'Tuesday':0})
data[['prem_day','success']].to_csv('gip1_2.csv')

### Вторая гипотеза 2 вариант(день недели премьеры фильма вновь выпущенного в прокат также влияет на сборы)

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=gip2_df.drop_duplicates(subset='movie_name'), x='prem_day',
              order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])

In [ ]:
## plt.figure(figsize=(10,5))
sns.boxplot(data=gip2_df.groupby('movie_name').sum()['sales_in_cu'].reset_index().merge(
    unique[['prem_day','movie_name']],how='inner',on='movie_name'),x='prem_day',y='sales_in_cu',showmeans=1,showfliers=0,
              order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=gip2_df.drop_duplicates(subset='movie_name'),x='prem_day',hue='success',
              order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
None

In [ ]:
data = gip2_df.groupby('movie_name').sum()['sales_in_cu'].reset_index().merge(
    unique[['prem_day','movie_name']],how='inner',on='movie_name')
data = pd.DataFrame({
    'Monday': data[data['prem_day']=='Monday']['sales_in_cu'],
    'Tuesday': data[data['prem_day']=='Tuesday']['sales_in_cu'],
    'Wednesday': data[data['prem_day']=='Wednesday']['sales_in_cu'],
    'Thursday': data[data['prem_day']=='Thursday']['sales_in_cu'],
    'Friday': data[data['prem_day']=='Friday']['sales_in_cu'],
    'Saturday': data[data['prem_day']=='Saturday']['sales_in_cu'],
    'Sunday': data[data['prem_day']=='Sunday']['sales_in_cu'],
})

data['Monday'] = data['Monday'].fillna(data['Monday'].median())
data['Tuesday'] = data['Tuesday'].fillna(data['Tuesday'].median())
data['Wednesday'] = data['Wednesday'].fillna(data['Wednesday'].median())
data['Thursday'] = data['Thursday'].fillna(data['Thursday'].median())
data['Friday'] = data['Friday'].fillna(data['Friday'].median())
data['Saturday'] = data['Saturday'].fillna(data['Saturday'].median())
data['Sunday'] = data['Sunday'].fillna(data['Sunday'].median())

print('Нулевая гипотеза:', '='.join(data))
print('Альтернативная гипотеза:', f'!({"=".join(data)})')
# общая средняя
grand_mean = data.values.flatten().mean()
# отклонение групповых средний от общей средней
ssb = sum(data[group].size * (group_mean - grand_mean)**2  for group, group_mean in data.mean().items())
# отклонения значений в внутри группы от средней группы
ssw = sum(sum((x - group_mean)**2 for x in data[group]) for group, group_mean in data.mean().items())

groups = data.shape[1]
dfb = groups - 1
dfw = data.size - groups
# межгрупповой средний квадрат  
mssb = ssb/dfb
# внутригрупповой средний квадрат
mssw = ssw/dfw

f_value = mssb/mssw

p = stats.f.sf(f_value, dfb, dfw)
print('Результат:')
if p < 0.05:
    print('отклоняем нулевую гипотезу')
else:
    print('НЕ отклоняем нулевую гипотезу')

In [ ]:
gip2 =gip2_df.groupby('movie_name').sum()['sales_in_cu'].reset_index().merge(unique[['prem_day','movie_name']],how='inner',on='movie_name')
groupa = gip2[gip2['prem_day'].isin(['Thursday','Friday'])]['sales_in_cu']
groupb = gip2[~gip2['prem_day'].isin(['Thursday','Friday'])]['sales_in_cu']
pvalue_1 = stats.ttest_ind(groupa, groupb,equal_var=0,alternative='greater')[1]
print('p-value для второй гипотезы:',pvalue_1)
print('Не отклоняем нулевую гипотезу' if pvalue_1 > 0.05 else 'Отклоняем нулевую гипотезу')

In [ ]:
#Среда - оптимальный день(хиквадрат тест)
data = gip2_df.drop_duplicates(subset='movie_name')
data['prem_day'] = data['prem_day'].map(
    {'Friday':0,'Wednesday':1, 'Thursday':0,'Monday':0,'Saturday':0,'Sunday':0,'Tuesday':0})
data[['prem_day','success']].to_csv('gip2_1.csv')

In [ ]:
#Четверг - оптимальный день(хиквадрат тест)
data = gip2_df.drop_duplicates(subset='movie_name')
data['prem_day'] = data['prem_day'].map(
    {'Friday':0,'Wednesday':0, 'Thursday':1,'Monday':0,'Saturday':0,'Sunday':0,'Tuesday':0})
data[['prem_day','success']].to_csv('gip2_2.csv')

In [ ]:
#Суббота - оптимальный день(хиквадрат тест)
data = gip2_df.drop_duplicates(subset='movie_name')
data['prem_day'] = data['prem_day'].map(
    {'Friday':0,'Wednesday':0, 'Thursday':0,'Monday':0,'Saturday':1,'Sunday':0,'Tuesday':0})
data[['prem_day','success']].to_csv('gip2_3.csv')